In [ ]:
import networkx as nx
import requests
import matplotlib as plt

In [ ]:
url = 'https://www.ia.pw.edu.pl/~mkamola/dataset-big/4.txt'
r = requests.get(url, allow_redirects=True)
open('4.txt', 'wb').write(r.content)

Zbadaj jaki jest rząd i rozmiar całej sieci: pierwotnej oraz po usunięciu pętli i duplikatów krawędzi (1);

In [ ]:

G_multi = nx.read_edgelist("4.txt", create_using=nx.MultiGraph)
G = nx.read_edgelist("4.txt", create_using=nx.Graph)

In [ ]:
I1 = nx.info(G_multi) 
print(I1 + '\n')
I2 = nx.info(G) 
print(I2 + '\n')

In [ ]:
nx.number_of_selfloops(G_multi)

In [ ]:
nx.number_of_selfloops(G)

In [ ]:
G_multi=nx.Graph(G_multi)

In [ ]:
G_multi.remove_edges_from(nx.selfloop_edges(G_multi))
G.remove_edges_from(nx.selfloop_edges(G))

In [ ]:
nx.number_of_selfloops(G_multi)

In [ ]:
nx.number_of_selfloops(G)

In [ ]:
N1 = nx.info(G_multi) 
print(N1 + '\n')
N2 = nx.info(G) 
print(N2 + '\n')

Wyodrębnij największą składową spójną, zbadaj jej rząd i rozmiar (1);

In [ ]:
nx.is_connected(G)

In [ ]:
nx.is_connected(G_multi)

In [ ]:
Set = [len(c) for c in sorted(nx.connected_components(G_multi), key=len, reverse=True)]

In [ ]:
largest_cc = max(nx.connected_components(G_multi), key=len)

In [ ]:
G_biggest=G_multi.subgraph(largest_cc)

In [ ]:
Inf = nx.info(G_biggest)
print(Inf)

In [ ]:
nx.number_of_nodes(G_biggest)

In [ ]:
nx.number_of_edges(G_biggest)

Wyznacz aproksymacje średniej długości ścieżki, operując na próbie losowej 100, 1000 i 10 tys. par wierzchołków (2);

In [ ]:
import random


In [ ]:
n_set = [100, 310, 620]
random_graph=[None] * len(n_set)
create_graph=[None] * len(n_set)
for i in n_set:
    random_graph = random.sample(G_biggest.nodes, i)
    create_graph = G_multi.subgraph(random_graph)
    lar_cc = max(nx.connected_components(create_graph), key=len)
    G_big=create_graph.subgraph(lar_cc)
    #print(nx.info(create_graph))
    print(nx.info(G_big))
    try:
        #aspl = nx.average_shortest_path_length(create_graph)
        aspl1 = nx.average_shortest_path_length(G_big)
        #print(i,aspl)
        print(i,aspl1)
    except nx.NetworkXError:
        print('Cannot count length')


In [ ]:
nx.average_shortest_path_length(G_biggest)

In [ ]:
def top_degree(graph, n):
    degree = nx.degree(graph)
    sdegree = sorted(degree,key=lambda n: -n[1])
    return sdegree[0:n]


In [45]:
top_degree(G_multi, 10)

[('net.databinder', 176),
 ('ui', 100),
 ('exo', 74),
 ('core', 53),
 ('geronimo', 51),
 ('org.scalatra', 51),
 ('com.github.philcali', 44),
 ('de.sciss', 44),
 ('org.scala-tools.testing', 42),
 ('maven', 36)]

In [46]:
top_degree(G_biggest,10)

[('ui', 100),
 ('core', 53),
 ('geronimo', 51),
 ('taglibs', 36),
 ('jboss', 32),
 ('org.eclipse.emf', 29),
 ('tomcat', 29),
 ('source', 20),
 ('infopop', 18),
 ('org.eclipse', 17)]

wyznacz liczbę rdzeni o największym możliwym rzędzie, o drugim możliwie największym rzędzie o trzecim możliwie największym rzędzie; jakie to są rzędy? (3);


In [58]:
from collections import Counter
core_numbers = nx.core_number(G_biggest)

#hist_data = sorted(Counter(core_numbers).items())
#hist_data = sorted(Counter(core_numbers))
#print(hist_data)
print(nx.info(nx.k_core(G_biggest,3)))
print(nx.info(nx.k_core(G_biggest,2)))
print(nx.info(nx.k_core(G_biggest,1)))

Name: 
Type: Graph
Number of nodes: 18
Number of edges: 39
Average degree:   4.3333
Name: 
Type: Graph
Number of nodes: 149
Number of edges: 265
Average degree:   3.5570
Name: 
Type: Graph
Number of nodes: 626
Number of edges: 742
Average degree:   2.3706


wykreśl rozkład stopni wierzchołków (1);


In [ ]:
import matplotlib.pyplot as plt
plt.scatter(*zip(*hist_data))


Częstosć stopni wierzchołków

In [ ]:
from collections import Counter
import matplotlib as plt
deg_dict = Counter(dlist)
degs = deg_dict.keys()
freqs = deg_dict.values()
plt.loglog(degs , freqs , 'bo')
plt.gcf().savefig('degree_freq.png')
plt.close()

Wykres rangowy

In [ ]:
cumul=[]
2 for i in range ( len (degs)):
3 cnt = 0
4 for j in range (i, len (degs)):
5 cnt = cnt + freqs[j]
6 cumul.append(cnt)
7 plt.loglog(degs , cumul , ’bo’)

wyznacz wykładnik rozkładu potęgowego metodą regresji dla dopełnienia dystrybuanty rozkładu stopni, dla przedziałów rozlokowanych logarytmicznie (3);


wyznacz wykres Hilla (3).

In [ ]:
import powerlaw
import numpy as np

NBINS = 50
bins = np.logspace(np.log10(min(dlist)),
                   np.log10(max(dlist)), num = NBINS)
bcnt, bedge = np.histogram(np.array(dlist),bins = bins)
alpha = np.zeros(len(bedge[:-2]))


for i in range(0,len(bedge)-2):
    fit=powerlaw.Fit(dlist,xmin=bedge[i], discrete=True)
    alpha[i]=fit.alpha

plt.semilogx(bedge[:-2], alpha)

Porównanie rozkładów

In [ ]:
fit = powerlaw.Fit(dlist, discrete=True)
print 'alpha (disc.) = ' + str(fit.alpha)
fig3 = fit.plot_ccdf(ax = fig3, color = 'r',
                     linestyle = '-')
fit.truncated_power_law.plot_ccdf(ax = fig3, color = 'g',
                                  linestyle = '-')
fit.lognormal.plot_ccdf(ax = fig3, color = 'b',
                        linestyle = '-')